It is a pain to have to keep dumping compute and garbage text files just to regenerate J each time.
The point of this notebook is to carefully review the process of J calculation,
so that we can perform it just once and save to the result to a shared numpy zip file.

In [1]:
%matplotlib notebook
import flux

Missing flux value for: GLEAM J202818-254302 at frequency: 204 MHz.
Missing flux value for: GLEAM J202818-254302 at frequency: 212 MHz.
Missing flux value for: GLEAM J202818-254302 at frequency: 220 MHz.
Missing flux value for: GLEAM J202818-254302 at frequency: 227 MHz.
Missing spectral index for: GLEAM J202818-254302


//anaconda3/envs/hera/lib/python3.7/site-packages/spin1_beam_model/jones_matrix_field.py:18: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  self.pos1_Elm_samples = h5f["pos1_Elm"].value
//anaconda3/envs/hera/lib/python3.7/site-packages/spin1_beam_model/jones_matrix_field.py:19: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  self.neg1_Elm_samples = h5f["neg1_Elm"].value
//anaconda3/envs/hera/lib/python3.7/site-packages/spin1_beam_model/jones_matrix_field.py:20: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  self.freq_nodes = h5f["frequencies"].value
//anaconda3/envs/hera/lib/python3.7/site-packages/spin1_beam_model/jones_matrix_field.py:21: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  self.L_model = h5f["spatial_bandlimit"].value
//anaconda3/envs/hera/lib/python3.7/site-packages/spin1_beam_model/jones_matrix_field.py:22: H5pyDeprec

delta_nu_in is 1000000.0


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import healpy as hp

In [3]:
list_az = []
list_alt = []
for source in flux.catalog.obj_catalog:
    ra = np.radians(source.ra_angle)
    dec = np.radians(source.dec_angle)
    az, alt = flux.rot.eq_to_topo(
        ra, dec,
        np.radians(flux.rot.hera_lat),
        flux.rot.get_lst(), radians=True
    )
    list_az.append(az)
    list_alt.append(alt)

In [4]:
"""
I am commenting this part out because we want to be able to
index into J as though we were indexing into obj_catalog.
list_az.sort()
list_alt.sort()
"""

# reformat into arrays so that the spline function can handle the single angles
list_az = np.array(list_az)
list_alt = np.array(list_alt)
print(list_az)
print(list_alt)

[-2.12387902  2.42830338  3.084854   ...  1.74785012  3.12247828
  1.97131093]
[ 1.48472446 -0.11997359 -0.62094507 ...  0.78010763 -0.65535201
  0.67708844]


In [5]:
#nside = 16, 32, 64, 128...
nside = 128

# healpix does the interpolation for you?
theta, phi = hp.pix2ang(nside, np.arange(12 * nside * nside))
az = phi
alt = np.pi / 2 - theta
J = flux.stokes.spline_beam_func(150e6, alt, az)

In [16]:
flux.stokes.spline_beam_func?

In [6]:
print(az[3])
print(list_az[3])
print(alt.dtype)
J.shape

5.497787143782138
-1.6642077558882762
float64


(196608, 2, 2)

In [7]:
print(az)
print(alt)
list_alt.dtype

[0.78539816 2.35619449 3.92699082 ... 2.35619449 3.92699082 5.49778714]
[ 1.56441744  1.56441744  1.56441744 ... -1.56441744 -1.56441744
 -1.56441744]


dtype('float64')

In [8]:
J = flux.stokes.spline_beam_func(150e6, list_alt, list_az)
J.shape

(3872, 2, 2)

In [9]:
empty_blocks = 0
for block in J:
    if block[0][0] == 0 and \
        block[1][0] == 0 and \
        block[0][1] == 0 and \
        block[1][1] == 0:
        empty_blocks += 1
empty_blocks

1517

In [10]:
""" This is a major problem, is it not? """
percent_empty = 100 * empty_blocks / len(J)
percent_empty

39.178719008264466

In [15]:
# Hard-coding a right-ascension,
# this should be a maximum response...

# the cold patch represents degrees 0 through 120

ra_max = 59.984791666666666
dec_max = -23.4345

J_max = flux.stokes.J_matrix(ra, dec, lst=np.pi / 3)
print(J_max)

[[[-0.0065094 -0.00757375j -0.01348645-0.02811096j]
  [-0.01682728-0.01411831j  0.00719214+0.01260143j]]]
